In [1]:
# =============================================================
# 04_evaluation.ipynb
# Évaluation du modèle Baseline et Fine-Tuning
# =============================================================

# ------------------------------------------------------------
# 1. Imports
# ------------------------------------------------------------
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import tensorflow as tf


In [3]:
# ------------------------------------------------------------
# 2. Chemins
# ------------------------------------------------------------
BASELINE_HISTORY_PATH = "../models/history_baseline.pkl"
FINETUNE_HISTORY_PATH = "../models/history_finetune_20251126-232904.pkl"
MODEL_PATH = "../models/inception_finetuned_20251126-232904.h5"
REPORTS_DIR = "../reports/"
os.makedirs(REPORTS_DIR, exist_ok=True)

DATASET_PATH = "../data/flower_images"
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 16

print("Dossiers vérifiés et chemins configurés.")


Dossiers vérifiés et chemins configurés.


In [4]:
# ------------------------------------------------------------
# 3. Charger les historiques
# ------------------------------------------------------------
with open(BASELINE_HISTORY_PATH, "rb") as f:
    history_baseline = pickle.load(f)

with open(FINETUNE_HISTORY_PATH, "rb") as f:
    history_finetune = pickle.load(f)

print("Historiques chargés.")


Historiques chargés.


In [5]:
# ------------------------------------------------------------
# 4. Tracer les courbes d'apprentissage Baseline vs Fine-Tuning
# ------------------------------------------------------------
def plot_compare_histories(hist1, hist2, title, filename):
    plt.figure(figsize=(12,5))

    # Accuracy
    plt.subplot(1,2,1)
    plt.plot(hist1["accuracy"], label="Train Baseline")
    plt.plot(hist1["val_accuracy"], label="Val Baseline")
    plt.plot(hist2["accuracy"], label="Train Fine-Tuning")
    plt.plot(hist2["val_accuracy"], label="Val Fine-Tuning")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy - " + title)
    plt.legend()
    plt.grid(True)

    # Loss
    plt.subplot(1,2,2)
    plt.plot(hist1["loss"], label="Train Baseline")
    plt.plot(hist1["val_loss"], label="Val Baseline")
    plt.plot(hist2["loss"], label="Train Fine-Tuning")
    plt.plot(hist2["val_loss"], label="Val Fine-Tuning")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss - " + title)
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    save_path = os.path.join(REPORTS_DIR, filename)
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Graphique sauvegardé : {save_path}")

plot_compare_histories(history_baseline, history_finetune, "Baseline vs Fine-Tuning", "compare_histories.png")


Graphique sauvegardé : ../reports/compare_histories.png


In [6]:
# ------------------------------------------------------------
# 5. Charger le modèle final Fine-Tuning
# ------------------------------------------------------------
model = tf.keras.models.load_model(MODEL_PATH)
print("Modèle Fine-Tuning chargé.")


Modèle Fine-Tuning chargé.


In [7]:
# ------------------------------------------------------------
# 6. Préparer le générateur pour toutes les images
# ------------------------------------------------------------
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

y_true = test_gen.classes
class_names = list(test_gen.class_indices.keys())


Found 5000 images belonging to 5 classes.


In [8]:
# ------------------------------------------------------------
# 7. Prédiction et matrice de confusion
# ------------------------------------------------------------
y_pred = model.predict(test_gen)
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names,
            annot_kws={"size":12})
plt.title("Matrice de Confusion - Fine-Tuning")
plt.xlabel("Prédictions")
plt.ylabel("Réel")
cm_path = os.path.join(REPORTS_DIR, "confusion_matrix.png")
plt.savefig(cm_path, dpi=300)
plt.close()
print("Matrice de confusion sauvegardée :", cm_path)


C:\Users\pc\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


313/313 ━━━━━━━━━━━━━━━━━━━━ 632s 2s/step
Matrice de confusion sauvegardée : ../reports/confusion_matrix.png


In [9]:
# ------------------------------------------------------------
# 8. Rapport de classification
# ------------------------------------------------------------
accuracy = accuracy_score(y_true, y_pred_classes)
report = classification_report(y_true, y_pred_classes, target_names=class_names)

report_path = os.path.join(REPORTS_DIR, "classification_report.txt")
with open(report_path, "w") as f:
    f.write(f"Accuracy globale : {accuracy:.4f}\n\n")
    f.write(report)

print("\nRapport de classification généré :")
print(f"Accuracy globale : {accuracy:.4f}")
print(report)
print("Fichier enregistré :", report_path)

print("\n🎉 Évaluation complète terminée !")



Rapport de classification généré :
Accuracy globale : 0.9826
              precision    recall  f1-score   support

       Lilly       0.98      0.96      0.97      1000
       Lotus       0.96      0.99      0.98      1000
      Orchid       0.98      0.98      0.98      1000
   Sunflower       1.00      1.00      1.00      1000
       Tulip       0.99      0.98      0.99      1000

    accuracy                           0.98      5000
   macro avg       0.98      0.98      0.98      5000
weighted avg       0.98      0.98      0.98      5000

Fichier enregistré : ../reports/classification_report.txt

🎉 Évaluation complète terminée !
